In [1]:
import os,csv,time,pylrc,shutil,requests,string,urllib,sys,re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

In [22]:
# (1)爬蟲wiki的英文縮寫統整表table，輸出檔案(1_df_contraction.xlsx)
def getWikiTable():
    # Wikipedia:List of English contractions (https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions)
    # https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722
    website_url = requests.get('https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions').text
    soup = BeautifulSoup(website_url,'lxml')
    table = soup.find('table',{'class':'wikitable sortable'})
    table_td = table.find_all('td')

    # 抓網頁裡的table
    L_contraction = []
    L_meaning = []
    for num in range(int(len(table_td)/2)):
        contraction = table_td[2*num].text.replace(' (rarely, cain\'t)','')
        meaning = table_td[2*num+1].text.replace('\n','').replace('[1]','').replace('[2]','').replace('[3]','').replace('[4]','').replace(' (colloquial)','').replace(' (archaic)','').replace(' (colloquial/Southern American English)','').replace(' (possessive forms of many nouns are homographic to this contraction)','')
        L_contraction.append(contraction)
        L_meaning.append(meaning)

    df_contraction = pd.DataFrame(list(zip(L_contraction,L_meaning)),columns=['Contraction','Meaning'])
    df_contraction.to_excel('table/singer_021-040/1_df_contraction_original.xlsx',index=0)
    return df_contraction

In [23]:
df_contraction = getWikiTable()
df_contraction

,Contraction,Meaning
0,'aight,alright
1,ain't,am not / is not / are not / has not / have not...
2,amn't,am not
3,aren't,are not
4,can't,cannot
...,...,...
141,you'll,you shall / you will
142,you're,you are/ you are
143,you've,you have
144,noun's,noun is


In [24]:
# (2)添加兩個欄位，Contraction2：首字大寫，Final：「'」取代為空白，並匯出檔案(2_df_contraction_edit.xlsx)
def editWikiTable(df_contraction):
    L_cap_x = []
    L_cap_x2 = []
    for i in range(len(df_contraction)):
        x = df_contraction['Contraction'][i]
        cap_x = string.capwords(x) # L_cap_x：新字串 (Contraction2欄位)
        cap_x2 = cap_x.replace("'"," ") # L_cap_x2：原始字串 (Final欄位)
        L_cap_x.append(cap_x)
        L_cap_x2.append(cap_x2)
    df_contraction['Contraction2'] = L_cap_x
    df_contraction['Original'] = L_cap_x2
    df_contraction.to_excel('table/singer_021-040/2_df_contraction_edit.xlsx',index=0)
    df_contraction_edit = df_contraction
    return df_contraction_edit,L_cap_x,L_cap_x2

In [25]:
df_contraction_edit,L_cap_x,L_cap_x2 = editWikiTable(df_contraction)
df_contraction_edit

,Contraction,Meaning,Contraction2,Original
0,'aight,alright,'aight,aight
1,ain't,am not / is not / are not / has not / have not...,Ain't,Ain t
2,amn't,am not,Amn't,Amn t
3,aren't,are not,Aren't,Aren t
4,can't,cannot,Can't,Can t
...,...,...,...,...
141,you'll,you shall / you will,You'll,You ll
142,you're,you are/ you are,You're,You re
143,you've,you have,You've,You ve
144,noun's,noun is,Noun's,Noun s


In [26]:
# (3)自定義建立縮寫替代的字典contractions_dict，L_cap_x2：原始字串，L_cap_x：新字串
# https://github.com/kootenpv/contractions
def create_contractions_dict(L_cap_x2, L_cap_x):
    contractions_dict = dict(zip(L_cap_x2, L_cap_x))
    #print(len(contractions_dict))
    return contractions_dict

In [27]:
contractions_dict = create_contractions_dict(L_cap_x2, L_cap_x)
contractions_dict

{' aight': "'aight",
 'Ain t': "Ain't",
 'Amn t': "Amn't",
 'Aren t': "Aren't",
 'Can t': "Can't",
 ' cause': "'cause",
 'Could ve': "Could've",
 'Couldn t': "Couldn't",
 'Couldn t ve': "Couldn't've",
 'Daren t': "Daren't",
 'Daresn t': "Daresn't",
 'Dasn t': "Dasn't",
 'Didn t': "Didn't",
 'Doesn t': "Doesn't",
 'Don t': "Don't",
 'D ye': "D'ye",
 'E er': "E'er",
 'Everybody s': "Everybody's",
 'Everyone s': "Everyone's",
 'Finna': 'Finna',
 'G day': "G'day",
 'Gimme': 'Gimme',
 'Giv n': "Giv'n",
 'Gonna': 'Gonna',
 'Gon t': "Gon't",
 'Gotta': 'Gotta',
 'Hadn t': "Hadn't",
 'Had ve': "Had've",
 'Hasn t': "Hasn't",
 'Haven t': "Haven't",
 'He d': "He'd",
 'He ll': "He'll",
 'He s': "He's",
 'He ve': "He've",
 'How d': "How'd",
 'Howdy': 'Howdy',
 'How ll': "How'll",
 'How re': "How're",
 'How s': "How's",
 'I d': "I'd",
 'I d ve': "I'd've",
 'I ll': "I'll",
 'I m': "I'm",
 'I m a': "I'm'a",
 'I m o': "I'm'o",
 'Innit': 'Innit',
 'I ve': "I've",
 'Isn t': "Isn't",
 'It d': "It'd",
 'It 

In [28]:
# (4)修改mp3特殊符號，如「’」、「´」等符號，並匯出檔案(3_df_mp3_1_modify_symbol.xlsx)
def modify_symbol(path):
    L_allFile = os.listdir(path)
    L_artist = [] # 暫存目前未刪除歌曲的歌手名稱
    L_title = [] # 暫存目前未刪除歌曲的歌曲檔名
    L_original_artist = []
    
    for file in L_allFile:
        if os.path.isdir(os.path.join(path,file)):
            L_artist.append(file)
            
            artistPath = path + '/' + file
            L_artistFile = os.listdir(artistPath)
            
            for file2 in L_artistFile:
                songPath = artistPath + '/' + file2
                if os.path.isfile(songPath):
                    
                    # 若遇到檔案有特殊符號，如「’」、「´」
                    new = file2.replace("’","'").replace("´","'").replace(" - The Voice Performance","").replace(" - Bonus Track","").replace(" - EP Version","")
                    mp3filename_edit = artistPath + '/' + new
                    os.rename(songPath,mp3filename_edit)
                    
                    L_title.append(new)
                    #print(artistPath + '/' + file2)
                    
                    # 原始歌手資料夾名稱
                    originalname = artistPath.split('/')[-1]
                    L_original_artist.append(originalname)
                    
    df_mp3_1_modify_symbol = pd.DataFrame(list(zip(L_original_artist,L_title)),columns=['ArtistName','SongName'])
    df_mp3_1_modify_symbol.to_excel('table/singer_021-040/3_df_mp3_1_modify_symbol.xlsx',index=0) # 匯出修改歌曲名稱的符號的xlsx
    return df_mp3_1_modify_symbol

In [29]:
path = r'D:/TuneFab Spotify Music Converter/songs2/singer_021-040' # mp3資料夾位址
df_mp3_1_modify_symbol = modify_symbol(path)
df_mp3_1_modify_symbol

,ArtistName,SongName
0,Charlie Puth,As You Are.mp3
1,Charlie Puth,Attention.mp3
2,Charlie Puth,BOY.mp3
3,Charlie Puth,Change.mp3
4,Charlie Puth,Dangerously.mp3
...,...,...
1550,Gary Barlow,Small Town Girls.mp3
1551,Gary Barlow,So Help Me Girl.mp3
1552,Gary Barlow,Superhero.mp3
1553,Gary Barlow,This House.mp3


In [30]:
# (5)修改mp3的檔案名稱，歌曲名稱部分應含有「'」符號，修改並補上「'」，並輸出含有新歌曲名稱的表(4_df_mp3_1_modify_symbol2.xlsx)
# 判斷字串是否包含子字串：https://blog.csdn.net/yl2isoft/article/details/52079960
# 子字串為自定義，採用建立contractions_dict的L_cap_x2陣列，迴圈一一搜尋
def editSongName(df_mp3_1_modify_symbol,L_cap_x2,df_contraction):
    #artist = 'Adele'
    #title = "Don t Stop"
    new_title_list = []
    
    for j in range(len(df_mp3_1_modify_symbol)):
        title = df_mp3_1_modify_symbol['SongName'][j]
        print(title)
        L_result = []
        
        for i in range(len(L_cap_x2)):
            result = title.find(L_cap_x2[i]) # 回傳為-1代表不存在，回傳不為-1代表存在
            L_result.append(result)
        #print(L_result)

        # 找到list中滿足條件的元素：https://blog.csdn.net/weixin_40995629/article/details/78715517
        # list找出滿足條件的元素位置：https://zhidao.baidu.com/question/488359074223051332.html
        selected = [i for i,v in enumerate(L_result) if v!=-1]
        #print(selected) # 印出符合條件的index值

        # 取代原來的歌曲名稱
        if len(selected) != 0:
            num = selected[0] # 取代的新字串在dict中的位置編號
            df_contraction_selected = df_contraction.loc[num] # 透過索引值選擇資料df.loc[[index],[colunm]]
            old_string = df_contraction_selected['Original']
            new_string = df_contraction_selected['Contraction2']
            print(old_string + ' >>> ' + new_string)
            new_title = title.replace(old_string,new_string)
            print(new_title)
            new_title_list.append(new_title)
        else:
            new_title_list.append(title)
            print('None Contraction Find!')
        print('-'*60)
    
    df_mp3_1_modify_symbol['SongNameNew'] = new_title_list
    df_mp3_1_modify_symbol.to_excel('table/singer_021-040/4_df_mp3_1_modify_symbol2.xlsx',index=0) # 修改歌曲名稱後的檔案
    df_mp3_1_modify_symbol_new = df_mp3_1_modify_symbol
    return new_title_list,df_mp3_1_modify_symbol_new

In [31]:
new_title_list,df_mp3_1_modify_symbol_new = editSongName(df_mp3_1_modify_symbol,L_cap_x2,df_contraction)
print(new_title_list)
df_mp3_1_modify_symbol_new

As You Are.mp3
None Contraction Find!
------------------------------------------------------------
Attention.mp3
None Contraction Find!
------------------------------------------------------------
BOY.mp3
None Contraction Find!
------------------------------------------------------------
Change.mp3
None Contraction Find!
------------------------------------------------------------
Dangerously.mp3
None Contraction Find!
------------------------------------------------------------
Done for Me.mp3
None Contraction Find!
------------------------------------------------------------
Empty Cups.mp3
None Contraction Find!
------------------------------------------------------------
How Long.mp3
None Contraction Find!
------------------------------------------------------------
If You Leave Me Now .mp3
None Contraction Find!
------------------------------------------------------------
LA Girls.mp3
None Contraction Find!
------------------------------------------------------------
Left Right Lef

None Contraction Find!
------------------------------------------------------------
Before I m Fool Enough.mp3
I m >>> I'm
Before I'm Fool Enough.mp3
------------------------------------------------------------
Beyond You.mp3
None Contraction Find!
------------------------------------------------------------
Blue Moon Of Kentucky.mp3
None Contraction Find!
------------------------------------------------------------
Carousel.mp3
None Contraction Find!
------------------------------------------------------------
Come Home Daddy.mp3
None Contraction Find!
------------------------------------------------------------
Coming Closer.mp3
None Contraction Find!
------------------------------------------------------------
Counterfeit Love.mp3
None Contraction Find!
------------------------------------------------------------
Counting Sheep Around the World.mp3
None Contraction Find!
------------------------------------------------------------
Cry Me A River.mp3
None Contraction Find!
----------

------------------------------------------------------------
Jingle Bells.mp3
None Contraction Find!
------------------------------------------------------------
Let S Put Our Heads Together.mp3
None Contraction Find!
------------------------------------------------------------
Limehouse Blues.mp3
None Contraction Find!
------------------------------------------------------------
Lonely Gondolier.mp3
None Contraction Find!
------------------------------------------------------------
Lonely Lane.mp3
None Contraction Find!
------------------------------------------------------------
Love Is On the Air Tonight.mp3
None Contraction Find!
------------------------------------------------------------
Lullaby Of Broadway.mp3
None Contraction Find!
------------------------------------------------------------
Lulu s Back In Town.mp3
 s >>> 's
Lulu's Back In Town.mp3
------------------------------------------------------------
Moonlight On Campus.mp3
None Contraction Find!
-----------------------

You ll >>> You'll
The Only Hand You'll Need to Hold.mp3
------------------------------------------------------------
The Only Way Out.mp3
None Contraction Find!
------------------------------------------------------------
The Pain of Loving You.mp3
None Contraction Find!
------------------------------------------------------------
The Party.mp3
None Contraction Find!
------------------------------------------------------------
The Power Of Love.mp3
None Contraction Find!
------------------------------------------------------------
The Right Combination.mp3
None Contraction Find!
------------------------------------------------------------
The River Unbroken.mp3
None Contraction Find!
------------------------------------------------------------
The Salt In My Tears.mp3
None Contraction Find!
------------------------------------------------------------
The Seeker.mp3
None Contraction Find!
------------------------------------------------------------
The Tracks Of My Tears.mp3
None Contra

['As You Are.mp3', 'Attention.mp3', 'BOY.mp3', 'Change.mp3', 'Dangerously.mp3', 'Done for Me.mp3', 'Empty Cups.mp3', 'How Long.mp3', 'If You Leave Me Now .mp3', 'LA Girls.mp3', 'Left Right Left.mp3', 'Losing My Mind.mp3', 'Marvin Gaye.mp3', 'My Gospel.mp3', 'One Call Away.mp3', 'Patient.mp3', 'See You Again.mp3', 'Slow It Down.mp3', 'Some Type of Love.mp3', 'Somebody Told Me.mp3', 'Suffer.mp3', 'The Way I Am.mp3', "Then There's You.mp3", 'Through It All.mp3', 'Up All Night.mp3', "We Don't Talk Anymore.mp3", 'A Case of You.mp3', 'A Change Is Gonna Come.mp3', 'A Song for You.mp3', 'Americano.mp3', 'Angel Eyes.mp3', 'Any Day Now.mp3', 'Back Pocket.mp3', 'Before You.mp3', 'Besame Mucho.mp3', "Don't Look at Me.mp3", "Don't Wanna Know.mp3", 'Drive.mp3', 'Feeling Good.mp3', 'I (Who Have Nothing).mp3', "I'm Blue, Skies.mp3", 'Mr Lonely Boy.mp3', 'Not Ready to Let You Go.mp3', 'Red Wine Is Good for My Heart.mp3', "She's Pretty, She Lies.mp3", "Somethin'stupid.mp3", 'Walkin my Baby Back Home.mp3

,ArtistName,SongName,SongNameNew
0,Charlie Puth,As You Are.mp3,As You Are.mp3
1,Charlie Puth,Attention.mp3,Attention.mp3
2,Charlie Puth,BOY.mp3,BOY.mp3
3,Charlie Puth,Change.mp3,Change.mp3
4,Charlie Puth,Dangerously.mp3,Dangerously.mp3
...,...,...,...
1550,Gary Barlow,Small Town Girls.mp3,Small Town Girls.mp3
1551,Gary Barlow,So Help Me Girl.mp3,So Help Me Girl.mp3
1552,Gary Barlow,Superhero.mp3,Superhero.mp3
1553,Gary Barlow,This House.mp3,This House.mp3


In [32]:
# (6)mp3檔案歌曲名稱重新命名
def rename_mp3file(path,df_mp3_1_modify_symbol_new):
    for i in range(len(df_mp3_1_modify_symbol_new)):
        artistname = df_mp3_1_modify_symbol_new['ArtistName'][i]
        current = df_mp3_1_modify_symbol_new['SongName'][i]
        new = df_mp3_1_modify_symbol_new['SongNameNew'][i]
        currentPath = path + '/' + artistname + '/' + current
        newPath = path + '/' + artistname + '/' + new
        try:
            os.rename(currentPath,newPath)
        except:
            pass
    print('Rename Done!')

In [33]:
rename_mp3file(path,df_mp3_1_modify_symbol_new)

Rename Done!


In [36]:
# (7)刪除歌曲名稱結尾為_0|_1|_2|_3|Remix|remix|Mix|mix|Live|live|Radio|radio... (重複歌曲+混音歌曲)，刪除後的檔案匯出(5_df_songlist.xlsx)
def mp3_select(path,df_mp3_1_modify_symbol_new):    
    string = '_0|_1|_2|_3|_4|_5|_6|_7|_8|Karaoke Version|Commentary|Reprise|reprise|Remastered|remastered|Remix|remix|Mono Version|Single Version|Instrumental|instrumental|Acoustic|acoustic|Radio Edit|Radio Version|Extended Version'
    # 要留下的
    df_mp3_2_mp3_select_Y = df_mp3_1_modify_symbol_new.loc[~df_mp3_1_modify_symbol_new['SongNameNew'].str.contains(string)].reset_index(drop=True)
    # 要刪除的
    df_mp3_2_mp3_select_N = df_mp3_1_modify_symbol_new.loc[df_mp3_1_modify_symbol_new['SongNameNew'].str.contains(string)].reset_index(drop=True)
    df_mp3_2_mp3_select_Y.to_excel('table/singer_021-040/5_df_mp3_2_mp3_select.xlsx',index=0) # 匯出Spotify下載的歌曲挑選後的結果
    
    # 使用df_mp3_2_mp3_select_N陣列刪除檔案
    for i in range(len(df_mp3_2_mp3_select_N)):
        artistname = df_mp3_2_mp3_select_N['ArtistName'][i]
        songname = df_mp3_2_mp3_select_N['SongNameNew'][i]
        songPath = path + '/' + artistname + '/' + songname
        #print(songPath)
        os.remove(songPath) # 刪除檔案

    return df_mp3_2_mp3_select_Y,df_mp3_2_mp3_select_N # 列出要留下的歌曲，列出要刪除的歌曲

In [37]:
df_mp3_2_mp3_select_Y,df_mp3_2_mp3_select_N = mp3_select(path,df_mp3_1_modify_symbol_new)
df_mp3_2_mp3_select_Y

,ArtistName,SongName,SongNameNew
0,Charlie Puth,As You Are.mp3,As You Are.mp3
1,Charlie Puth,Attention.mp3,Attention.mp3
2,Charlie Puth,BOY.mp3,BOY.mp3
3,Charlie Puth,Change.mp3,Change.mp3
4,Charlie Puth,Dangerously.mp3,Dangerously.mp3
...,...,...,...
1550,Gary Barlow,Small Town Girls.mp3,Small Town Girls.mp3
1551,Gary Barlow,So Help Me Girl.mp3,So Help Me Girl.mp3
1552,Gary Barlow,Superhero.mp3,Superhero.mp3
1553,Gary Barlow,This House.mp3,This House.mp3


In [38]:
df_mp3_2_mp3_select_N

,ArtistName,SongName,SongNameNew


In [39]:
# (8)將歌手和歌名整理並輸出表格(df_songlist)和檔案(6_df_songlist.xlsx)
def mp3_finallist(df_mp3_2_mp3_select_Y):
    mp3_original_L = []
    mp3_artist_L = []
    mp3_song_L = []
    
    for i in range(len(df_mp3_2_mp3_select_Y)):
        artist = df_mp3_2_mp3_select_Y['ArtistName'][i]
        song = df_mp3_2_mp3_select_Y['SongNameNew'][i]
        
        mp3_original_L.append(artist)
        artist_list = artist.split(',')
        mp3_artist_L.append(artist_list[0]) # 針對資料夾有多位歌手處理，取第一個為主要演唱者
        song = song.replace('.mp3','')
        mp3_song_L.append(song)
    
    # 輸出datafrme(df_songlist)和xlsx(6_df_songlist.xlsx)
    df_songlist = pd.DataFrame(list(zip(mp3_original_L,mp3_artist_L,mp3_song_L)),columns=['Original','Artist','Title'])
    df_songlist.to_excel('table/singer_021-040/6_df_songlist.xlsx',index=0)
    return df_songlist

In [40]:
df_songlist = mp3_finallist(df_mp3_2_mp3_select_Y)
df_songlist

,Original,Artist,Title
0,Charlie Puth,Charlie Puth,As You Are
1,Charlie Puth,Charlie Puth,Attention
2,Charlie Puth,Charlie Puth,BOY
3,Charlie Puth,Charlie Puth,Change
4,Charlie Puth,Charlie Puth,Dangerously
...,...,...,...
1550,Gary Barlow,Gary Barlow,Small Town Girls
1551,Gary Barlow,Gary Barlow,So Help Me Girl
1552,Gary Barlow,Gary Barlow,Superhero
1553,Gary Barlow,Gary Barlow,This House


___

In [24]:
name = ['123(Remix)','Tom_5','Jason','Marry(live)','Julia','854-Version',
        '123Instrument','test_0 version','test acoustic','good','123(remix)']
number = ['a','b','c','d','e','f','g','h','i','j','k']
stringtest = '_0|_1|_2|_3|_4|_5|_6|Remastered|remastered|Remix|remix|Mix|mix|Single Version|Instrumental|instrumental|Acoustic|acoustic|(Live)|(live)'
dftest = pd.DataFrame(list(zip(name,number)),columns=['NAME','NUMBER'])
dftest2 = dftest.loc[~dftest['NAME'].str.contains(stringtest)]
dftest2

,NAME,NUMBER
0,123(Remix),a
1,Tom_5,b
2,Jason,c
3,Marry(live),d
4,Julia,e
5,854-Version,f
6,123Instrument,g
7,test_0 version,h
8,test acoustic,i
9,good,j


In [21]:
artist = 'Adele'
title = "Don t Say"
L_result = []
new_title_list = []
for i in range(len(L_cap_x2)):
    result = title.find(L_cap_x2[i]) # 回傳為-1代表不存在，回傳不為-1代表存在
    L_result.append(result)
#print(L_result)

# 找到list中滿足條件的元素：https://blog.csdn.net/weixin_40995629/article/details/78715517
# list找出滿足條件的元素位置：https://zhidao.baidu.com/question/488359074223051332.html
selected = [i for i,v in enumerate(L_result) if v!=-1]
#print(selected) # 印出符合條件的index值

# 取代原來的歌曲名稱
if len(selected) != 0:
    num = selected[0] # 取代的新字串在dict中的位置編號
    df_contraction_selected = df_contraction.loc[num] # 透過索引值選擇資料df.loc[[index],[colunm]]
    old_string = df_contraction_selected['Original']
    new_string = df_contraction_selected['Contraction2']
    print(old_string + ' >>> ' + new_string)
    new_title = title.replace(old_string,new_string)
    print(new_title)
    new_title_list.append(new_title)
else:
    new_title_list.append(title)
    print('None Contraction Find!')

Don t >>> Don't
Don't Say


In [9]:
# (8)存在於df_songlist的歌曲(代表有下載到歌詞的)，複製歌曲到另一個資料夾(songs_edit)
def copyfile(base1,base2,df_songlist):
    oldname_list = [] # 在songs2資料夾的原始檔名
    currentname_list = [] # 複製到songs_edit資料夾的檔名
    newname_list = [] # 欲修改的新檔名
    for i in range(len(df_songlist)):
        original = df_songlist['Original'][i]
        artist = df_songlist['Artist'][i]
        title = df_songlist['Title'][i]
        
        filename = title + '.mp3'
        #base1 = r'D:/TuneFab Spotify Music Converter/songs2/'
        oldpath = base1 + original + '/' + filename # 原始音檔位址
        oldname_list.append(oldpath)
        #base2 = r'D:/TuneFab Spotify Music Converter/songs_edit/'
        newpath = base2 + filename # 複製目的地
        currentname_list.append(newpath)
        
        newname = base2 + artist + '_' + filename # 複製後的新檔名，後面renamefile()執行會使用到
        newname_list.append(newname)
        
        if os.path.exists(oldpath): # 音檔存在
            message = 'exists'
            shutil.copy(oldpath,newpath)
            print(message + ' - ' + newpath)
        else:
            message = 'not exist' # 音檔不存在
            print(message)
    
    df_copyfile = pd.DataFrame(list(zip(oldname_list,currentname_list,newname_list)),columns=['OldName','CurrentName','NewName'])
    return df_copyfile

In [10]:
base1 = r'D:/TuneFab Spotify Music Converter/songs2/'
base2 = r'D:/TuneFab Spotify Music Converter/songs_edit/'
df_copyfile = copyfile(base1,base2,df_songlist)
df_copyfile

exists - D:/TuneFab Spotify Music Converter/songs_edit/18.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Airplanes.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Amnesia.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Babylon.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Beside You.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Better Man.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Broken Home.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Castaway.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Catch Fire.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Close As Strangers.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Disconnected.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Don t Stop.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Empty Wallets.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/End Up Here.mp3
exists - D:/TuneFab Sp

exists - D:/TuneFab Spotify Music Converter/songs_edit/Stride (Jump On The Fizzy) featuring No Secrets (feat  No Secrets).mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Everything My Heart Desires.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Give Me Your Heart.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Good Times.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Heart And Soul.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Hold On To Our Love.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/I Breathe Again.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/I Can t Live Without Your Love.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/The Best Thing.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Time Is On Our Side.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/Touch Me.mp3
exists - D:/TuneFab Spotify Music Converter/songs_edit/You Make Me Believe In Love.mp3
exists - D:/T

,OldName,CurrentName,NewName
0,D:/TuneFab Spotify Music Converter/songs2/5 Se...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
1,D:/TuneFab Spotify Music Converter/songs2/5 Se...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
2,D:/TuneFab Spotify Music Converter/songs2/5 Se...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
3,D:/TuneFab Spotify Music Converter/songs2/5 Se...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
4,D:/TuneFab Spotify Music Converter/songs2/5 Se...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
...,...,...,...
134,D:/TuneFab Spotify Music Converter/songs2/Adel...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
135,D:/TuneFab Spotify Music Converter/songs2/Adel...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
136,D:/TuneFab Spotify Music Converter/songs2/Agne...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
137,D:/TuneFab Spotify Music Converter/songs2/Agne...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...


In [11]:
# (9)資料夾(songs_edit)的歌曲重新命名，依據「歌手_歌名」格式命名
def renamefile(df_copyfile):
    for i in range(len(df_copyfile)):
        old = df_copyfile['OldName'][i]
        current = df_copyfile['CurrentName'][i]
        new = df_copyfile['NewName'][i]
        try:
            os.rename(current,new)
        except:
            pass
    print('Rename Done!')

In [12]:
renamefile(df_copyfile)

Rename Done!
